# Annika Carlson
## AEEM5042 - Module 4 Assignment

In [149]:
import numpy as np
import sys, os
sys.path.append(os.getcwd() + r"/..")
from IPython.display import display, Markdown
from scipy.optimize import fsolve
from utils.tbm import shockRelation
from Project0.compflow import compflow

### Question 1

Set given/known values:

In [150]:
M0 = 0.8        # freestream mach number
A0_A1 = 0.92    # capture to highlight area ratio

Initial Assumptions:

In [151]:
y = 1.4        # specific heat ratio, cold

Additional assumptions are made later on.

Find:

a) Pressure coefficient, $C_p$, at the stagnation point

b) Inlet Mach number, $M_1$

c) Lip contraction ratio, $A_1/A_{th}$, for a throat Mach $M_{th}$ = 0.75, assuming $p_t/p_{t,th}$ = 1

d) Diffuser area ratio, $A_2/A_{th}$ if $M_2$ = 0.5 and $p_t/p_{t,th}$ = 0.98

e) The non-dimensional inlet additive drag, $D_{add}/\left( p_0 \cdot A_1 \right)$

#### a)

Start by getting the equation for pressure coefficient, $C_p$, in terms of freestream mach number using pressure isentropic relations.

$$
C_p = \frac{2\left(P_{t0} - P_0\right)}{\gamma \cdot P_0 \cdot M_0^2}
$$

where $P_{t0}$ is freestream total pressure, and $P_0$ is freestream static pressure. From isentropic relations:

$$
\frac{P_{t0}}{P_0} = \left(1 + \frac{\gamma - 1}{2}*M_0^2\right)^{\frac{\gamma}{\gamma - 1}} \Rightarrow P_{t0} = P_0 \left(1 + \frac{\gamma - 1}{2}*M_0^2\right)^{\frac{\gamma}{\gamma - 1}}
$$

Plugging this into the $C_p$ equation for $P_{t0}$ and simplifying, we get:

$$
C_p = \frac{2\left[P_0 \left(1 + \frac{\gamma - 1}{2}*M_0^2\right)^{\frac{\gamma}{\gamma - 1}} - P_0\right]}{\gamma \cdot P_0 \cdot M_0^2} = \frac{2 \cdot P_0\left[\left(1 + \frac{\gamma - 1}{2}*M_0^2\right)^{\frac{\gamma}{\gamma - 1}} - 1\right]}{\gamma \cdot P_0 \cdot M_0^2}
$$

$$
\Rightarrow C_p = \frac{2 \left[\left(1 + \frac{\gamma - 1}{2}*M_0^2\right)^{\frac{\gamma}{\gamma - 1}} - 1\right]}{\gamma \cdot M_0^2} 
$$

This leaves us $C_p$ in terms of $\gamma$ and $M_0$, which we can now solve.

In [152]:
Cp = (2*((1 + ((y - 1)/2)*(M0**2))**(y/(y - 1)) - 1))/(y*(M0**2))
display(Markdown(f"$C_p$ = ${Cp:.2f}$"))

$C_p$ = $1.17$

#### b)

To find inlet Mach number, $M_1$, we can simply use the area ratio $A_0/A_1$ and $M_0$.

In [153]:
# Use fsolve to solve the area ratio equation for M1
def equation(M1):
    return (M1/M0)*((1 + ((y - 1)/2)*M0**2)/(1 + ((y - 1)/2)*M1**2))**((y + 1) / (2 * (y - 1))) - A0_A1

M1_guess = 0.5
M1 = fsolve(equation, M1_guess)[0]
display(Markdown(f"$M_1$ = ${M1:.2f}$"))

$M_1$ = $0.66$

#### c)

To solve for lip contraction ratio, we use the area relation with $M_1$ and $M_{th}$.

Assume:

$M_{th}$ = 0.75

$P_{t1}/P_{t,th}$ = 1

In [154]:
Mth = 0.75          # throat Mach number
Pt1_Ptth = 1        # inlet to throat total pressure ratio

A1_Ath = (1/Pt1_Ptth)*(Mth/M1)*((1 + ((y - 1)/2)*M1**2)/(1 + ((y - 1)/2)*Mth**2))**((y + 1)/(2*(y - 1)))
display(Markdown(f"$A_1/A_{{th}}$ = ${A1_Ath:.2f}$"))

$A_1/A_{th}$ = $1.06$

#### d)

To solve for diffuser area ratio, we again use the area relation equation but with $M_2$ and $M_{th}$.

Assume:

$M_2$ = 0.5

$P_{t2}/P_{t,th}$ = 0.98

In [155]:
M2 = 0.5            # engine fan face Mach number
Pt2_Ptth = 0.98     # throat to engine fan face total pressure ratio

A2_Ath = (1/Pt2_Ptth)*(Mth/M2)*((1 + ((y - 1)/2)*M2**2)/(1 + ((y - 1)/2)*Mth**2))**((y + 1)/(2*(y - 1)))
display(Markdown(f"$A_2/A_{{th}}$ = ${A2_Ath:.2f}$"))

$A_2/A_{th}$ = $1.29$

#### e)

Now find nondimensional inlet additive drag using the equation with $M_0$, $M_1$, and $\gamma$.

In [156]:
Dadd_p0A1 = y*M0*((1 + ((y - 1)/2)*M0**2)/(1 + ((y - 1)/2)*M1**2))**(y/(2*(y - 1)))*((M1/M0)* \
        ((1 + ((y - 1)/2)*M1**2)/(1 + ((y - 1)/2)*M0**2))**0.5 - 1) + ((1 + ((y - 1)/2)*M0**2)/(1 + ((y - 1)/2)*M1**2))**(y/(y - 1)) - 1

display(Markdown(f"$D_{{add}}/p_0A_1$ = ${Dadd_p0A1:.2f}$"))

$D_{add}/p_0A_1$ = $-0.09$

### Question 2

Set given/known values:

In [157]:
M0 = 0.2            # freestream Mach number 
M2 = 0.65           # fan face Mach number
Pt2_Pt0 = 0.98      # inlet total pressure recovery

Assumptions:

In [158]:
y = 1.4         # specific heat ratio, cold

Find:

Capture-to-engine face area ratio, $A_0/A_2$

In [159]:
# solve the area ratio using Pt2/Pt0, M0 and M2

A0_A2 = Pt2_Pt0*(M2/M0)*((1 + ((y - 1)/2)*M0**2)/(1 + ((y - 1)/2)*M2**2))**((y + 1)/(2*(y - 1)))
display(Markdown(f"$A_0/A_2$ = ${A0_A2:.2f}$"))

$A_0/A_2$ = $2.56$

### Question 3

Set given/known values:

In [160]:
M0 = 1.6            # flight Mach number
A0_A1 = 0.90        # inlet capture area ratio
A2_A1 = 1.2         # diffuser area ratio
Pt2_Pt1 = 0.95      # engine face to inlet total pressure loss

Assumptions:

Stream tube area is constant from stations 0 to 1.

In [161]:
y = 1.4

Find:

a) Inlet Mach number, $M_1$

b) Inlet total pressure recovery, $\pi_d = P_{t2}/P_{t0}$

#### a)

From normal shock tables, we can find mach number after the normal shock given that $M_0 = M_x = 1.6$.

From the shock tables, $M_y = 0.67$

Now we can use the inlet capture area ratio to find $M_1$, now using $M_y$ in place of $M_0$.

In [162]:
M0y = 0.67

# Use fsolve to solve the area ratio equation for M1
def equation(M1):
    return (M1/M0y)*((1 + ((y - 1)/2)*M0y**2)/(1 + ((y - 1)/2)*M1**2))**((y + 1) / (2 * (y - 1))) - A0_A1

M1_guess = 0.67
M1 = fsolve(equation, M1_guess)[0]
display(Markdown(f"$M_1$ = ${M1:.2f}$"))

$M_1$ = $0.56$

#### b)

In order to find $\pi_d$, we can use $P_{t2}/P_{t1}$ and $P_{t1}/P_{t0}$ to find $P_{t2}/P_{t0}$.

We know that $P_{t1}/P_{t0}$ is the same as the pressure ratio across the shock, $P_y/P_x$, which can be found from the shock tables for $M_0$ = 1.6.

From the tables, $P_y/P_x$ = 0.8952

Now, we find simply that $\pi_d = \frac{P_{t2}}{P_{t0}} = \frac{P_{t2}}{P_{t1}} \cdot \frac{P_{t1}}{P_{t0}}$

In [163]:
Pty_Ptx = 0.8952

Pt2_Pt0 = Pt2_Pt1*Pty_Ptx
display(Markdown(f"$\pi_d$ = ${Pt2_Pt0:.2f}$"))

$\pi_d$ = $0.85$

### Question 4

Set given/known values:

In [164]:
M0 = 2.5            # freestream Mach number
theta1 = 8.0        # first ramp angle
theta2 = 12.0       # second ramp angle

Assumptions:

In [165]:
y = 1.4     # specific heat ratio, cold

Find:

a) Total pressure recovery of the inlet shock system, $\pi_d$

b) Compare the total pressure recovery to a normal shock (only) inlet at the same Mach number

#### a)

Start by finding the shock angle, $\beta_1$, for the first oblique shock using the theta-beta-Mach relation. Then we can find the normal component of the freestream Mach number.

In [166]:
B1 = shockRelation(M=M0, theta=theta1, y=y)
M0n = M0*np.sin(np.deg2rad(B1))
display(Markdown(f"normal freestream mach number, $M_{{0n}}$ = ${M0n:.2f}$"))

normal freestream mach number, $M_{0n}$ = $1.25$

Now we can use the normal component, $M_{0n}$, and solve as a normal shock to get $M_{1n}$ and $P_{t1}/P_{t0}$ for the first oblique shock from the normal shock tables.

From the normal shock tables: $M_{1n} = 0.81$,  $P_{t1}/P_{t0} = 0.987057$

Now solve for M1, the actual Mach number after the first oblique shock.

In [167]:
M1n = 0.81
Pt1_Pt0 = 0.987057

M1 = M1n/np.sin(np.deg2rad(B1) - np.deg2rad(theta1))
display(Markdown(f"mach number after shock 1, $M_1$ = ${M1:.2f}$"))

mach number after shock 1, $M_1$ = $2.16$

Now, we can solve across the second oblique shock the same way.

In [168]:
B2 = shockRelation(M=M1, theta=theta2, y=y)
M1n = M1*np.sin(np.deg2rad(B2))
display(Markdown(f"normal mach number after shock 1, $M_{{1n}}$ = ${M1n:.2f}$"))

normal mach number after shock 1, $M_{1n}$ = $1.35$

From the normal shock tables: $M_{2n} = 0.76$,  $P_{t2}/P_{t1} = 0.969737$

In [169]:
M2n = 0.76
Pt2_Pt1 = 0.969737

M2 = M2n/np.sin(np.deg2rad(B2) - np.deg2rad(theta2))
display(Markdown(f"mach number after shock 2, $M_2$ = ${M2:.2f}$"))

mach number after shock 2, $M_2$ = $1.70$

Now solve across the final normal shock.

From the normal shock tables: $M_3 = 0.64$,  $P_{t3}/P_{t2} = 0.855721$

Lastly, solve for total pressure recovery of the system, $P_{t3}/P_{t0}$

In [170]:
Pt3_Pt2 = 0.855721
Pt3_Pt0 = Pt1_Pt0*Pt2_Pt1*Pt3_Pt2
display(Markdown(f"$\pi_d$ = ${Pt3_Pt0:.2f}$"))

$\pi_d$ = $0.82$

#### b)

If the inlet only had one normal shock, we would just solve for the pressure recovery using the freestream Mach number, $M_0 = 2.5$, across a normal shock.

From the normal shock tables: $M_1 = 0.51$,  $P_{ty}/P_{tx} = 0.499015$

Thus, $\pi_d = 0.50$.

This means that with a single normal shock at the inlet, there would be about a 50% pressure loss from the freestream to the fan face of the engine, compared to only about an 18% loss with the configuration of oblique and normal shocks shown for the same Mach number.

### Question 5

Set given/known values:

In [171]:
Ae_Ath = 2.4    # exit to throat area ratio
Pt = 100       # nozzle inlet pressure (kPa)

Assumptions:

The nozzle is choked at the throat $\left(M_{th} = 1\right)$, and isentropic until the shock.

In [172]:
y = 1.33       # specific heat ratio, hot

Find:

a) Ambient pressure, $P_0$ $\quad$ Note: $\left(P_0 = P_y\right)$

b) Exhaust temperature, $T_y$

c) Mass flow rate through the nozzle

#### a)

Since we are assuming the nozzle is choked, we can use $Ae/Ath = A/A* = 2.4$ to look at the compressible flow tables for $\gamma = 1.33$ to find the supersonic solution for Mach number at the exit as well as the pressure ratio $P_e/P_te$.

From the compressible flow tables: $M_e = 2.34$,  $P_e/P_{te} = 0.074704$

And then we can solve for the ambient pressure, $P_0$, by first finding the pressure at the exit before the shock, $P_e$.

In [173]:
Pe_Pet = 0.074704
Pe = Pe_Pet*Pt
display(Markdown(f"Pressure at the exit, $P_e$ = $P_x$ = ${Pe:.2f}$ $kPa$"))

Pressure at the exit, $P_e$ = $P_x$ = $7.47$ $kPa$

Now we can solve for the pressure across the shock at the exit using the normal shock tables for $M_x = M_e = 2.34$.

From the normal shock tables: $M_y = 0.52$,  $P_y/P_x = 6.109483$

And now we can use the pressure ratio across the shock to find $P_y$, which is $P_0$.

In [174]:
P0_Pe = 6.109483
P0 = Pe*P0_Pe
display(Markdown(f"$P_0$ = ${P0:.2f}$ $kPa$"))

$P_0$ = $45.64$ $kPa$

#### b)

Assume:

Since the nozzle is isentropic until the shock, the total temperature will be constant from the throat until right before the shock. Thus:

In [175]:
Ttth = 350 + 273.15      # throat total temperature (K)
Ttx = Ttth               # total temperature before the shock (K)

To find the exhaust temperature after the shock, $T_y$, we can start with using the compressible flow tables for $\gamma = 1.33$ at the Mach number before the shock, $M_e = M_x$.

From the compressible flow tables: $T_x/T_{tx} = 0.525355$

And from the normal shock tables for across the shock: $T_y/T_x = 1.823030$

These can now both be used to solve for $T_y$.

In [176]:
Tx_Ttx = 0.525355
Ty_Tx = 1.823030

Tx = Tx_Ttx*Ttx
Ty = Ty_Tx*Tx
display(Markdown(f"$T_y$ = ${Ty:.2f}$ $K$"))

$T_y$ = $596.81$ $K$

#### c)

Assume:

$A_{th}$ = 0.25

In [177]:
Ath = 0.25      # throat area (m^2)

To find the mass flow rate, we can use the Mass Flow Parameter using the compressible flow code from project 0.

In [178]:
MFP = compflow(1.0, y)[4]
mdot = MFP*Ath*(Pt*1000)/np.sqrt(287*Ttth)
display(Markdown(f"$\dot{{m}}$ = ${mdot:.2f}$ $kg/s$"))

$\dot{m}$ = $39.76$ $kg/s$

### Question 6

Set given/known values:

Assumptions:

ADD

Find:

All dimensions dimensions including throat diameter ($D_8$), exit diameter ($D_9$), divergence angle ($\alpha$), length of divergent section ($L_s$) 

and $C_{fg}$, $F_g$, $C_V$